# Magnetic System Explorer

Comprehensive notebook for magnetic system analysis with real-time visualization.
Supports any magnetic material with interactive simulation and phase diagram generation.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from spinlab.analysis.visualization import SpinVisualizer
from spinlab.core.spin_system import SpinSystem
from spinlab.core.hamiltonian import Hamiltonian
from ase.io import read
from ase.build import bulk, make_supercell
import time

# Configure matplotlib for notebook
%matplotlib inline

# Initialize visualizer
viz = SpinVisualizer()
print("Magnetic System Explorer Ready!")

Magnetic System Explorer Ready!


In [ ]:
# System parameters - MODIFY THESE FOR YOUR ANALYSIS
##################

# Structure parameters
MATERIAL = 'Fe'               # Material symbol for ASE
CRYSTAL = 'bcc'               # Crystal structure
LATTICE_PARAM = 2.87          # Lattice parameter (Å)
n_cells = 8                   # Supercell size (8x8x1)
SYSTEM_SIZE = (n_cells, n_cells, 1)

# Create ASE structure
structure = bulk(MATERIAL, CRYSTAL, a=LATTICE_PARAM, cubic=True)
structure = structure.repeat(SYSTEM_SIZE)

print(f"Created structure: {len(structure)} atoms ({MATERIAL} {CRYSTAL})")

# Exchange interactions (eV)
J1 = -0.02                    # 1st neighbor exchange
J2 = 0.0                      # 2nd neighbor exchange
J3 = 0.0                      # 3rd neighbor exchange

# Anisotropic exchange (eV) - for 1st neighbors only
Jxx = 0.0; Jyy = 0.0; Jzz = 0.0
Jxy = 0.0; Jxz = 0.0; Jyz = 0.0

# Single-ion anisotropy (eV)
A = 0.0                       # Easy-axis anisotropy

# External fields
g = 2.0                       # g-factor
gamma = 0.0                   # Electric coupling (e·Å)
B_z = 0.0                     # Magnetic field (Tesla)
E_z = 0.0                     # Electric field (V/Å)

# Neighbor cutoffs (Å) - you still need to define these
cutoff_distance = 3.5         # 1st neighbor cutoff
second_cutoff = 4.9           # 2nd neighbor cutoff  
third_cutoff = 6.1            # 3rd neighbor cutoff

# Simulation parameters
temperature = 100             # Temperature (K)
spin_magnitude = 1.0          # Spin magnitude
model_type = '3D'             # 'Ising', 'XY', '3D'
angular_res = 1.0             # Angular resolution (degrees)

# Monte Carlo parameters
MC_STEPS = 1000
UPDATE_INTERVAL = 50

print(f"Parameters set: J1={J1}, T={temperature}K, Model={model_type}")

In [ ]:
# Create comprehensive Hamiltonian
##################

# Optional: Define magnetic species for multi-component systems
# magnetic_species = ['Fe', 'Ni']  # Only consider these atoms as magnetic
magnetic_species = None  # None = all atoms are magnetic

hamiltonian = Hamiltonian(magnetic_species=magnetic_species)

# Filter structure if magnetic species are specified
if magnetic_species is not None:
    structure, index_map = hamiltonian.filter_magnetic_atoms(structure)
    print(f"Using magnetic species filtering: {magnetic_species}")
else:
    index_map = None
    print(f"All atoms considered magnetic")

# Add exchange interactions for multiple shells
if J1 != 0.0:
    hamiltonian.add_exchange(J=J1, neighbor_shell="shell_1", name="J1_exchange")
if J2 != 0.0:
    hamiltonian.add_exchange(J=J2, neighbor_shell="shell_2", name="J2_exchange")
if J3 != 0.0:
    hamiltonian.add_exchange(J=J3, neighbor_shell="shell_3", name="J3_exchange")

# Add anisotropic exchange (using Kitaev terms for directional coupling)
if any([Jxx, Jyy, Jzz]) != 0.0:
    hamiltonian.add_kitaev(
        K_couplings={"x": Jxx, "y": Jyy, "z": Jzz},
        neighbor_shell="shell_1",
        name="anisotropic_exchange"
    )

# Add single-ion anisotropy
if A != 0.0:
    hamiltonian.add_single_ion_anisotropy(K=A, axis=[0, 0, 1], name="easy_axis")

# Add magnetic field (Zeeman term)
if B_z != 0.0:
    hamiltonian.add_magnetic_field(B_field=[0, 0, B_z], g_factor=g, name="zeeman")

# Add electric field coupling
if E_z != 0.0:
    hamiltonian.add_electric_field(E_field=[0, 0, E_z], gamma=gamma, name="electric")

print("Hamiltonian terms added:")
for i, name in enumerate(hamiltonian.term_names):
    print(f"  {i+1}. {name}")

In [ ]:
# Create SpinSystem
##################
system = SpinSystem(
    structure=structure,
    hamiltonian=hamiltonian,
    magnetic_model=model_type.lower(),
    spin_magnitude=spin_magnitude
)

# Setup neighbor shells
##################
# Note: You still need to define cutoff distances manually
# SpinLab automatically creates "shell_1", "shell_2", etc. based on cutoff order
cutoffs = [cutoff_distance]
if J2 != 0.0:
    cutoffs.append(second_cutoff)
if J3 != 0.0:
    cutoffs.append(third_cutoff)

neighbors = system.get_neighbors(cutoffs)

print("Neighbor shells setup:")
for shell, neighbor_array in neighbors.items():
    avg_neighbors = np.mean(np.sum(neighbor_array >= 0, axis=1))
    print(f"  {shell}: {neighbor_array.shape} (avg {avg_neighbors:.1f} neighbors/site)")

# Initialize spin configuration based on model type
##################
if model_type.lower() == 'ising':
    # Ising: ±Z spins only - need to implement this method
    print("Ising model: initializing ±Z spins")
    # For now, use random and constrain to ±Z
    system.random_configuration()
    # Constrain to ±Z directions
    system.spin_config[:, :2] = 0  # Zero x,y components
    system.spin_config[:, 2] = np.sign(system.spin_config[:, 2]) * spin_magnitude
elif model_type.lower() == 'xy':
    # XY: spins in xy-plane
    print("XY model: initializing xy-plane spins")
    system.random_configuration()
    # Constrain to xy-plane
    system.spin_config[:, 2] = 0  # Zero z component
    # Normalize xy components
    xy_norm = np.linalg.norm(system.spin_config[:, :2], axis=1, keepdims=True)
    system.spin_config[:, :2] = system.spin_config[:, :2] / xy_norm * spin_magnitude
else:
    # 3D: random 3D configuration
    print("3D model: initializing random 3D spins")
    system.random_configuration()

# System information
##################
initial_energy = system.calculate_energy()
initial_magnetization = system.calculate_magnetization()

print(f"\nSystem created: {len(system.positions)} magnetic sites")
print(f"Initial energy: {initial_energy:.4f} eV")
print(f"Initial |M|: {np.linalg.norm(initial_magnetization):.3f}")
print(f"Spin configuration shape: {system.spin_config.shape}")
print(f"Energy per site: {initial_energy/len(system.positions):.6f} eV")

## Initial Spin Configuration

Visualize the random initial spin state:

In [ ]:
# Plot initial random configuration using actual positions
viz.plot_spin_configuration(
    system.positions,
    system.spin_config,
    title=f"{MATERIAL} Initial Random Configuration",
    figsize=(8, 8)
)